# Introduction

Welcome to the **Distil Labs** hands‑on tutorial.  
In this walkthrough you will:

1. **Specialize** a small language model (SLM) with Distil Labs.
2. **Build** a fully local retrieval‑augmented‑generation (RAG) pipeline that uses your fine‑tuned model.

By the end you’ll have a domain‑aware assistant you can run entirely on your own hardware.

# Notebook Setup

##### Install ollama

To install ollama, follow the instructions in https://ollama.com/download. If you are on a linux platform the following command should get the job done but otherwise, the download page should cover the installation

In [ ]:
! curl -fsSL https://ollama.com/install.sh | sh

Once ollama is installed, we should start the application. If you installed ollama using the installer, make sure the app is running. If you installed ollama with with CLI, you can start the deamon with `ollama serve`, for example using `nohup` to make sure it stays in the bacgkround.

In [ ]:
! nohup ollama serve

Make sure the app is running by executing 

In [ ]:
! ollama list

##### Install python libraries

In [ ]:
! pip install langchain-core langchain_community langchain-openai langchain-huggingface langchain-ollama
! pip install wikipedia pandas numpy requests rich pyyaml rouge_score ollama

In [ ]:
%env TOKENIZERS_PARALLELISM=false

# Step 1: Understand your data

Before we can specialize a model or build a retrieval‑augmented generation (RAG) pipeline, we need to **inspect the knowledge source** we’ll be working with. In this tutorial, our task is: **answer questions about the Roman Empire**.

_Why bother looking at the raw data first?_  
• It clarifies the scope (what’s _in_ and what’s _out_ of domain).  
• It helps us spot formatting issues or noisy sections.  
• It lets us craft realistic evaluation questions early on.

### Retrieve a reference article
To keep things quick, we’ll use a single reference document: the English **Wikipedia** page for the Roman Empire.
In a production system you’d likely combine multiple sources, but one page is enough to demo the workflow.

In [ ]:
import wikipedia, textwrap

# Disable auto-suggest so we get the exact page
page = wikipedia.page("Roman Empire", auto_suggest=False)
wikipedia_text = page.content

print("First 1200 characters of the article:\n")
print(textwrap.fill(wikipedia_text[:1200], 110))


### Sample questions we want to answer
Let’s jot down a few questions that our finished system should handle. Capturing these **early** gives us a mini test‑set for later.

In [ ]:
sample_qa = [
    {"question": "Who was the first emperor of the Roman Empire?", "answer": "Augustus (formerly Octavian)"},
    {"question": "When did the Roman Empire collapse?", "answer": "The west Roman Empire fell in 476 CE and the Easy laster until the fall of Constantinopol in 1453."},
    {"question": "Which two languages were most widely spoken across the empire?", "answer": "Latin in the West and Greek in the East"},
    {"question": "What monumental arena in Rome hosted gladiatorial games?", "answer": "The Colosseum"},
]

for qa in sample_qa:
    print(f"Q: {qa['question']}\nA: {qa['answer']}\n")


# Step 2: Specialize a question answering model with Distil Labs

In this chapter we will fine‑tune a small language model with **distil labs** in three steps:

1. **Data upload** – upload the job description, train/test splits, unstructured contexts, and a YAML config.
2. **Teacher evaluation** – verify that a large model can solve the task; its accuracy becomes the benchmark.
3. **SLM training** – distil the teacher’s behaviour into a 135 M‑parameter student.

### Authentification

In [ ]:
import json
import os
import requests


def distil_bearer_token(DL_USERNAME: str, DL_PASSWORD: str) -> str:
    response = requests.post(
        "https://cognito-idp.eu-central-1.amazonaws.com",
        headers={
            "X-Amz-Target": "AWSCognitoIdentityProviderService.InitiateAuth",
            "Content-Type": "application/x-amz-json-1.1",
        },
        data=json.dumps({
            "AuthParameters": {
                "USERNAME": DL_USERNAME,
                "PASSWORD": DL_PASSWORD,
            },
            "AuthFlow": "USER_PASSWORD_AUTH",
            "ClientId" : "4569nvlkn8dm0iedo54nbta6fd",
        })
    )
    response.raise_for_status()
    return response.json()["AuthenticationResult"]["AccessToken"]


DL_USERNAME="DL_USERNAME"
DL_PASSWORD="DL_PASSWORD"

AUTH_HEADER = {"Authorization": distil_bearer_token(DL_USERNAME, DL_PASSWORD)}
print("Success")

### Data Upload

The data for this example should be stored in the data_location directory. Lets first take a look at the current directory to make sure all files are available. Your current directory should look like:
```
├── README.md
├── rag-tutorial.ipynb
└── data
    ├── job_description.json
    ├── test.csv
    ├── train.csv
    └── unstructured.csv
```

#### Job Description
A job description explains the question answering task in plain english and follows the general structure below:

In [ ]:
import json
from pathlib import Path
import rich.json

with open(Path("data").joinpath("job_description.json")) as fin:
    rich.print(rich.json.JSON(fin.read()))

#### Train/test set

We need a small train dataset to begin distil labs training and a testing dataset that we can use to evaluate the performance of the fine-tuned model. Here, we use the train and test datasets from the data_location directory where each is a CSV file with below 100 (question, answer) pairs.

In [ ]:
from pathlib import Path
from IPython.display import display

import pandas as pd

print("# --- Train set")
train = pd.read_csv(Path("data").joinpath("train.csv"))
display(train)

print("# --- Test set")
test = pd.read_csv(Path("data").joinpath("test.csv"))
display(test)

#### Unstructured dataset
The unstructured dataset is used to guide the teacher model in generating diverse, domain-specific data. Here, we will use the chunks of the wikipedia article as the unstructured data for our problem.

In [ ]:
import pandas as pd
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Split the document into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_text(wikipedia_text)

# Save the documents into local storage
contexts_dataframe = pd.DataFrame([{"context": split} for split in all_splits])
contexts_dataframe.to_csv("data/unstructured.csv")
contexts_dataframe

#### Data upload

In [ ]:
import json

import requests
import yaml
from pathlib import Path

# Specify the config
config = {
    "base": {"task": "question-answering-open-book"},
    "setup": {"student_model_name": "HuggingFaceTB/SmolLM2-135M-Instruct"},
    "synthgen": {
        "data_generation_strategy": "qa-open-book",
    },
    "tuning": {
        "num_train_epochs": 8,
    },
}

# Package your data
data_dir = Path("data")
data = {
    "job_description.json": open(data_dir / "job_description.json", encoding="utf-8").read(),
    "train.csv": open(data_dir / "train.csv", encoding="utf-8").read(),
    "test.csv": open(data_dir / "test.csv", encoding="utf-8").read(),
    "unstructured.csv": open(data_dir / "unstructured.csv", encoding="utf-8").read(),
    "config.yaml": yaml.dump(config),
}

# Upload data
response = requests.post(
    "https://api.distillabs.ai/uploads",
    data=json.dumps(data),
    headers={"content-type": "application/json", **AUTH_HEADER},
)
upload_id = response.json().get("id")
print(f"Upload successful. ID: {upload_id}")


### Teacher Evaluation
Before training an SLM, distil labs validates whether a large language model can solve your task:

In [ ]:
from pprint import pprint

# Start teacher evaluation
response = requests.post(
    f"https://api.distillabs.ai/teacher-evaluations/{upload_id}",
    headers=AUTH_HEADER
)

teacher_evaluation_id = response.json().get("id")
pprint(response.json())

Poll the status endpoint until it completes, then inspect the accuracy. If accuracy is unsatisfactory, refine the job description.


In [ ]:
response = requests.get(
    f"https://api.distillabs.ai/teacher-evaluations/{teacher_evaluation_id}/status",
    headers=AUTH_HEADER
)
pprint(response.json())

### SLM Training
Once the teacher evaluation completes successfully, start the SLM training:

In [ ]:
import time
from pprint import pprint

# Start SLM training
response = requests.post(
    f"https://api.distillabs.ai/trainings/{upload_id}",
    headers=AUTH_HEADER,
)

pprint(response.json())
slm_training_job_id = response.json().get("id")

We can analyze the status of the training job using the `jobs` API. The following code snippets displays the current status of the job we started before. When the job is finished (`status=complete`), we can use the `jobs` API again to get the benchmarking result - the accuracy of the LLM and the accuracy of the fine-tuned SLM. We can achieve this using:

In [ ]:
import json
from pprint import pprint

response = requests.get(
    f"https://api.distillabs.ai/trainings/{slm_training_job_id}/status",
    headers=AUTH_HEADER,
)
pprint(response.json())

When the job is finished (`status=complete`), we can use the `jobs` API again to get the benchmarking result - the accuracy of the LLM and the accuracy of the fine-tuned SLM. We can achieve this using:

In [ ]:
from pprint import pprint

response = requests.get(
    f"https://api.distillabs.ai/trainings/{slm_training_job_id}/evaluation-results",
    headers=AUTH_HEADER,
)

pprint(response.json())


### Download Your Model
Once training is complete, download your model for deployment.

In [ ]:
# Get model download URL
response = requests.get(
    f"https://api.distillabs.ai/trainings/{slm_training_job_id}/model",
    headers=AUTH_HEADER
)

pprint(response.json())


# Step 3: Build a local RAG system with your fine‑tuned model

Now that we have a small language model fine‑tuned specifically for Roman‑Empire question‑answering, we can build our RAG pipeline around it. This domain‑specialized LLM will provide more accurate, context‑aware answers than our baseline model while still running entirely on local hardware. The main objectives for us are as follows:
  * Launch a lightweight chat model locally with **ollama**.
  * Chunk a Wikipedia article, embed the chunks with **HuggingFace sentence‑transformers**, and store them in an **in‑memory vector store**.
  * Glue retrieval and generation together in a minimal **RAG class**, then test the loop end‑to‑end.

### Register and test the downloaded model 
Once your model is trained, it should be unpacked and registered with ollama. The downloaded model directory already contains everything that is needed and the model can be registed with the command below. Once it is ready, we can test the model with a standard OPenAI interface

In [ ]:
! ollama create model-distillabs -f model/Modelfile

In [ ]:
from openai import OpenAI

client = OpenAI(
    base_url = 'http://localhost:11434/v1',
    api_key='ollama', # required, but unused
)

response = client.chat.completions.create(
  model="model-distillabs",
  messages=[
    {"role": "user", "content": "When did the roman empire collapse?"},
  ]
)
print(response.choices[0].message.content)

### Index our target dataset

This section walks through loading the **Wikipedia article on the Roman Empire** into an in‑memory vector store (adapted from [https://python.langchain.com/docs/tutorials/rag/](https://python.langchain.com/docs/tutorials/rag/)):

In [ ]:
from langchain_community.document_loaders import WikipediaLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore

# Split the document into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
text_splits = text_splitter.split_text(wikipedia_text)
document_splits = text_splitter.create_documents(text_splits)

# Embed and index the chunks
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L12-v2")
vector_store = InMemoryVectorStore(embeddings)
indexed = vector_store.add_documents(documents=document_splits)

### Define the RAG logic

Now that our dataset is indexed and the chat model is live, we can **wire retrieval and generation together**. In this section we implement a bite‑sized `RAG` helper class that

1. fetches the top‑k passages most similar to the user’s question,
2. feeds those passages and the question into the language model via a structured prompt, and
3. returns a concise answer.

With this plumbing in place, answering a question becomes a single‑function call.

In [ ]:
from langchain_core.documents import Document
from langchain_openai import ChatOpenAI
from langchain_core.vectorstores import InMemoryVectorStore


class RAG:
    def __init__(self, vector_store: InMemoryVectorStore, llm: ChatOpenAI):
        self.vector_store = vector_store
        self.llm = llm

        self.SYSTEM_PROMPT = """
You are a problem solving model working on task_description XML block:

<task_description>
Answer the question using information from one of the context paragrpahs.
The answer should should contain all important information from the context paragraph but stay short, one sentence maximum.
</task_description>

You will be given a single task with context in the context XML block and the task in the question XML block
Solve the task in question block based on the context in context block.
Generate only the answer, do not generate anything else
"""

        self.PROMPT_TEMPLATE = """
Now for the real task, solve the task in question block based on the context in context block.
Generate only the solution, do not generate anything else.

<context>
{context}
</context>

<question>
{question}
</question>
"""

    def retrieve(self, question: str, k: int = 3):
        return self.vector_store.similarity_search(question, k=k)

    def generate(self, question: str, context_docs):
        context = "\n\n".join(doc.page_content for doc in context_docs)
        messages = [
            {"role": "system", "content": self.SYSTEM_PROMPT},
            {"role": "user", "content": self.PROMPT_TEMPLATE.format(context=context, question=question)},
        ]
        return self.llm.invoke(messages).content

    def answer(self, question: str):
        return self.generate(question, self.retrieve(question))

### Plug the new model into RAG
With the fine‑tuned weights now running locally, the last step is to introduce the specialized LLM into our existing RAG helper class. The retrieval component fetches the most relevant passages about the Roman Empire—while the generation step leverages a model that has been trained on our domain‑specific data.

In [ ]:
from langchain_openai import ChatOpenAI

tuned_llm = ChatOpenAI(
    base_url='http://localhost:11434/v1',
    api_key="EMPTY",
    model="model-distillabs"
)
tuned_rag = RAG(vector_store=vector_store, llm=tuned_llm)
print(tuned_rag.answer("When did the roman empire collapse?"))

### Test our RAG system

In [ ]:
import pandas as pd
import numpy as np
from rouge_score import rouge_scorer


rouge = rouge_scorer.RougeScorer(["rougeL"], use_stemmer=True)
test_dataset = pd.read_csv("data/test.csv")

results = []
for idx, example in test_dataset.iterrows():
    reference = example["answer"]
    prediction = tuned_rag.answer(example["question"])
    metric = rouge.score(
        target=reference,
        prediction=prediction,
    )
    results.append(metric["rougeL"].fmeasure)

print(f"Aggregate metric: {np.mean(results)} +/- {np.std(results)}")
# Aggregate metric: 0.253503946795671 +/- 0.16689695917279943